In [1]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2  # Importar regularización L2 desde keras.regularizers
%matplotlib inline

### Cargar Datos

In [2]:

with open('entrenamiento.p', 'rb') as file:
    training_data = pickle.load(file)
    

with open('prueba.p', 'rb') as file:
    test_data = pickle.load(file)
    

with open('validacion.p', 'rb') as file:
    validation_data = pickle.load(file)


### Dividir la data en X, Y. Entrenamiento, prueba y validación

In [3]:
X_train = training_data["features"]
y_train = training_data["labels"]

In [4]:
X_test = test_data["features"]
y_test = test_data["labels"]

In [5]:
X_val = validation_data["features"]
y_val = test_data["labels"]

### Normalizar los datos, debido a que los datos están con valores de 0 a 255 al dividirlos por 255 esto nos da un porcentaje de que tanto se acerca a 255.

In [6]:
X_train = X_train/255
X_test = X_test/255
X_val = X_val/255

In [7]:
y_train.shape

(34799,)

### Cargar los datos de entrenamiento y como son labels volverlos en categoricos desde 0 hasta 42, es decir 43 categorias diferentes

In [8]:
y_cat_train = to_categorical(y_train, 43)


In [9]:
y_cat_test = to_categorical(y_test, 43)

In [10]:
y_cat_val = to_categorical(y_val, 43)

# Armado del modelo

In [11]:
modelo = Sequential()

## Primer conjunto de capas

### Capa convolucional

Se crea una capa convulucional que tiene 32 filtros. Debido a que nuestras imagenesposeen un canal de colres, la forma del input sería de 32, 32, 3 siendo 3 el canal de los colores. Asimismo, el kernel sería de 4 por 4 lo cual quiere decir que se tomaran cuadrados de 4 por 4 pixeles para la capa de convolusión. Esto tendrá como output 32 imagenes diferentes, debido a los 32 filtros cada una de ellas con un tamaño de 29 (tamaño de imagen de entrada - (tamaño de kernel) + 1). 

In [12]:
modelo.add(Conv2D(filters = 32, kernel_size = (4, 4), input_shape = (32, 32, 3), activation = 'relu',))

### Capa de submuestreo (max-pooling)

Se crea una capa que realiza maxpooling. Este método lo que hace es crear una ventana del tamaño que se pida, en nuestro caso es de 2x2. En esa ventana de  4x4 se toma el valor del pixel más alto, es decir el valor maximo dentro de la ventana y luego se crea otra "imagen" de valores con ese nuevo valor. Esto se realiza con cada cuadro (2x2) de la imágen input ingresada. El output de esto es una imagen de 14 x 14. Esto se obtuvo a través de: floor((29-poolsize)/stride)+1. Ej. floor((29-2)/2) +1 => floor(27/2) +1 => floor(13.5)+1 => 13+1 = 14

In [13]:
modelo.add(MaxPool2D(pool_size = (2, 2)))

## Segundo conjunto de capas

Debido a que ahora se estan trabajando con colores. La cantidad de datos a procesar son demasiados por lo cual vale la pena realizar otra capa. Tanto convolucional como de submuestreo antes de analizar los datos a través de una red neuronal. 

### Capa convolucional 

In [14]:
modelo.add(Conv2D(filters = 32, kernel_size = (4, 4), input_shape = (32, 32, 3), activation = 'relu',))

### Capa de sub-muestreo (Pooling)

In [15]:
modelo.add(MaxPool2D(pool_size = (2, 2)))

### Flatten

In [16]:
modelo.add(Flatten())

## 256 neuronas para la capa escondida con función de activación de Relu

In [17]:
modelo.add(Dense(256, activation = 'relu'))

### 43 outputs finales debido a que se tienen 43 categorías diferentes. Esta última capa tiene la función de decisión (activación) de Softmax

In [18]:
modelo.add(Dense(43, activation = 'softmax'))

# Compilación del modelo

### Para la compilación del modelo se utilizara la función de perdida de categorical crossentropy. Esta función sirve cuando se tiene una clasificación multi clase el cual es nuestro caso al tener 43 posibles decisiones.

In [19]:
modelo.compile(loss = 'categorical_crossentropy',
              optimizer = 'rmsprop',
              metrics = ['accuracy'])

In [20]:
modelo.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 29, 29, 32)        1568      
                                                                 
 max_pooling2d (MaxPooling2  (None, 14, 14, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 32)        16416     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 5, 5, 32)          0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 800)               0         
                                                                 
 dense (Dense)               (None, 256)               2

# Entrenamiento del modelo

In [21]:
# Crear un generador de imágenes con aumento de datos
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [22]:
# Aumentar el tamaño del lote y el número de épocas
batch_size = 128
epochs = 30

modelo = Sequential()
modelo.add(Conv2D(filters=32, kernel_size=(3, 3), input_shape=(32, 32, 3), activation='relu'))
modelo.add(MaxPool2D(pool_size=(2, 2)))
modelo.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
modelo.add(MaxPool2D(pool_size=(2, 2)))
modelo.add(Flatten())
modelo.add(Dense(512, activation='relu'))
modelo.add(Dropout(0.5))
modelo.add(Dense(43, activation='softmax'))

In [23]:
modelo.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])

In [24]:
modelo.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 15, 15, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_3 (Conv2D)           (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 6, 6, 64)          0         
 g2D)                                                            
                                                                 
 flatten_1 (Flatten)         (None, 2304)              0         
                                                                 
 dense_2 (Dense)             (None, 512)              

In [25]:
X_train = X_train / 255
X_test = X_test / 255
X_val = X_val / 255

y_cat_train = to_categorical(y_train, 43)
y_cat_test = to_categorical(y_test, 43)
y_cat_val = to_categorical(y_val, 43)


In [26]:
modelo = Sequential()
modelo.add(Conv2D(filters=32, kernel_size=(3, 3), input_shape=(32, 32, 3), activation='relu'))
modelo.add(MaxPool2D(pool_size=(2, 2)))
modelo.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
modelo.add(MaxPool2D(pool_size=(2, 2)))
modelo.add(Flatten())
modelo.add(Dense(512, activation='relu', kernel_regularizer=l2(0.01)))
modelo.add(Dropout(0.5))
modelo.add(Dense(43, activation='softmax'))

### Se compila el modelo especificando la función de pérdida (categorical_crossentropy, adecuada para clasificación multiclase), el optimizador (rmsprop en este caso) y las métricas que se desea rastrear durante el entrenamiento.

In [27]:
modelo.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [28]:
modelo.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 15, 15, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 6, 6, 64)          0         
 g2D)                                                            
                                                                 
 flatten_2 (Flatten)         (None, 2304)              0         
                                                                 
 dense_4 (Dense)             (None, 512)              

# Evaluación del Modelo

In [29]:
# Evaluar el modelo en el conjunto de prueba
test_metrics = modelo.evaluate(X_test, y_cat_test)

395/395 [==============================] - 1s 3ms/step - loss: 12.1426 - accuracy: 0.0116


In [30]:
# Obtener predicciones del modelo en el conjunto de prueba
y_pred = modelo.predict(X_test)

395/395 [==============================] - 1s 3ms/step


In [31]:
!pip3 install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.


In [32]:
# Convertir las predicciones a etiquetas (índices de clase)
y_pred_labels = np.argmax(y_pred, axis=1)

In [33]:
# Mostrar un informe de clasificación que incluye precisión, recall y f1-score para cada clase
class_report = classification_report(np.argmax(y_cat_test, axis=1), y_pred_labels)
print(class_report)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        60
           1       0.00      0.00      0.00       720
           2       0.00      0.00      0.00       750
           3       0.03      0.15      0.06       450
           4       0.00      0.00      0.00       660
           5       0.00      0.00      0.00       630
           6       0.00      0.00      0.00       150
           7       0.06      0.02      0.03       450
           8       0.02      0.04      0.02       450
           9       0.00      0.00      0.00       480
          10       0.00      0.00      0.00       660
          11       0.02      0.01      0.01       420
          12       0.00      0.00      0.00       690
          13       0.00      0.00      0.00       720
          14       0.00      0.00      0.00       270
          15       0.00      0.00      0.00       210
          16       0.00      0.00      0.00       150
          17       0.00    

/Users/priscillagonzalez/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/priscillagonzalez/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/priscillagonzalez/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, m

# Mejoras y experimentación

### - Se podrían ajustar mejor algunas parámetros como lo es la tasa de aprendizaje, el batch size y las épocas que se manejan
### - Se podría utilizar regularización como L1 o L2 para evitar el sobreajuste
### - También se podría cambiar el tamaño de la imagen para poder observar cómo es que afecta el rendimiento